In [1]:
import chess.pgn
import chess.engine
import multiprocessing
import io

def analyze_single_game(game_data):
    game_text, engine_path = game_data
    print("Analyzing a game...")  # Logging for debugging

    # Parse the game from PGN text
    game = chess.pgn.read_game(io.StringIO(game_text))
    if game is None:
        print("Game is None")
        return None  # Handle empty or invalid games

    board = game.board()

    # Load the chess engine
    try:
        engine = chess.engine.SimpleEngine.popen_uci(engine_path)
    except Exception as e:
        print(f"Engine failed to start: {e}")
        return None
    # Configure the engine
    engine.configure({'Threads': 1, 'Hash': 128})

    # Extract Elo ratings and result
    white_elo = int(game.headers.get("WhiteElo", "0"))
    black_elo = int(game.headers.get("BlackElo", "0"))
    result = game.headers.get("Result", "*")

    move_number = 1
    evaluations = []

    # Go through each move
    for move in game.mainline_moves():
        # Get the SAN notation before pushing the move
        move_san = board.san(move)

        # Apply the move to the board
        board.push(move)

        # Get Stockfish evaluation with depth limit
        try:
            info = engine.analyse(board, chess.engine.Limit(depth=12))
        except Exception as e:
            print(f"Engine analysis failed: {e}")
            engine.quit()
            return None

        eval_score = info["score"].relative

        if eval_score.is_mate():
            eval_value = f"Mate in {eval_score.mate()}"
        else:
            eval_value = eval_score.score() / 100  # Convert centipawns to pawns

        # Collect the move and evaluation
        evaluations.append({
            'move_number': move_number,
            'move': move_san,
            'evaluation': eval_value
        })

        move_number += 1

    engine.quit()

    return {
        'white_elo': white_elo,
        'black_elo': black_elo,
        'result': result,
        'evaluations': evaluations
    }

def read_games_from_pgn(pgn_file):
    games = []
    with open(pgn_file) as pgn:
        while True:
            game = chess.pgn.read_game(pgn)
            if game is None:
                break
            # Convert the game back to PGN text
            exporter = chess.pgn.StringExporter(headers=True, variations=False, comments=False)
            game_text = game.accept(exporter)
            games.append(game_text)
    return games

def analyze_pgn_file(pgn_file, engine_path):
    # Read all games from the PGN file
    games_text = read_games_from_pgn(pgn_file)
    # Prepare data for multiprocessing
    game_data_list = [(game_text, engine_path) for game_text in games_text]

    if not game_data_list:
        print("No games found in the PGN file.")
        return []

    # Determine the number of processes to use
    cpu_count = multiprocessing.cpu_count()
    num_processes = min(cpu_count, len(game_data_list))

    print(f"Starting analysis with {num_processes} processes...")  # Logging

    # Use multiprocessing to analyze games in parallel
    with multiprocessing.Pool(processes=num_processes) as pool:
        results = pool.map(analyze_single_game, game_data_list)

    return results

if __name__ == '__main__':
    multiprocessing.set_start_method('spawn')  # Important for Windows

    # Paths to the Stockfish engine and PGN file
    stockfish_path = r"C:\Users\foivo\Downloads\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe"
    pgn_file = "example2.pgn"

    # Analyze the PGN file
    games_analysis = analyze_pgn_file(pgn_file, stockfish_path)

    # Example: Print the analysis of the first game
    if games_analysis:
        first_game = games_analysis[0]
        if first_game is not None:
            print(f"White Elo: {first_game['white_elo']}, Black Elo: {first_game['black_elo']}, Result: {first_game['result']}")
            for move_eval in first_game['evaluations']:
                print(f"Move {move_eval['move_number']}: {move_eval['move']}, Evaluation: {move_eval['evaluation']}")
        else:
            print("First game analysis returned None.")
    else:
        print("No game analysis was performed.")


Starting analysis with 2 processes...
